Warp Radolan data to GTIFF   
https://docs.wradlib.org/en/stable/georef.html#raster-functions  

Use this cell to create geotiff in gk3 or ll projection
change line 77 and 83 with correct coordinates and crs

In [ ]:
#For Files located in single folder
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 
from osgeo import osr


#Working dir paths must be clean otherwise you have to make list comprehension that only bin files in files list
wdir = "/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/"
out = "out"
#Change to Workingdir
os.chdir(wdir)

if not os.path.exists(out):
    os.mkdir(out)


#create the necessary Spatial Reference Objects for the RADOLAN-projection 
proj_stereo = wrl.georef.create_osr("dwd-radolan") #create osr object
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)
radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)

# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()
proj_gk3.ImportFromEPSG(31467) #dhdn3

#Bug handling
epsg_31467 = wrl.georef.epsg_to_osr(31467) 
epsg_4326 = wrl.georef.epsg_to_osr(4326) 


# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                       projection_source=proj_wgs,
                                       projection_target=proj_gk3)

#print(radolan_grid_gk)
#Get list of files inside the folder
files = os.listdir()
print(len(files)," files inside the folder")

#1 because of out is first file later do a list comprehension maybe more beautiful ^^
for file in files[1:]:
    proj_stereo = wrl.georef.create_osr("dwd-radolan") #create osr object
    proj_wgs = osr.SpatialReference()
    proj_wgs.ImportFromEPSG(4326)
    radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)
    #proj_gk3 = osr.SpatialReference()
    #proj_gk3.ImportFromEPSG(31467) #dhdn3
    radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
    #radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                      # projection_source=proj_wgs,
                                       #projection_target=proj_gk3)
    #print("Load: ",file ," data, georeferencing, reprojection and translate to Gtif...")
    data_raw = None
    meta = None
    data = None
    xy = None
    ds_gk3 = None
    data_raw, meta  = wrl.io.read_radolan_composite(file)
    data_raw = data_raw.round(decimals=3)
    data_raw = data_raw.astype('float32')
    #print(radolan_grid_ll[1])
    #print(data_raw.dtype)
    #print(np.median(data_raw))
    #flip the data and the coords up-down (from lower left to upper left corner)
    #data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_xy, 'upper')
    data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_ll, 'upper')
    #print(xy[-1])
    #print(xy)
    #print(radolan_grid_gk)
    #Create raster dataset: Geotiff with radolan projection
    #ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_stereo, nodata=-9999)
    ds_gk3 = wrl.georef.create_raster_dataset(data, xy, proj_wgs, -9999) # https://docs.wradlib.org/en/stable/generated/wradlib.georef.raster.create_raster_dataset.html
    #Reproject dataset to GK3
    #ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=proj_gk3, size=(900,900)) # align=(3052995.645672334, 6067805.270329222) spacing=1000 size=(900,900)
    #Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
    #This line is needed until fixed
    #ds.SetProjection(epsg_31467.ExportToWkt())
    #ds.SetProjection(epsg_4326.ExportToWkt())
    #write raster dataset
    filename = file[15:25] + ".tif"
    wrl.io.write_raster_dataset(wdir + out + "/" + filename, ds_gk3, "GTiff")
    #print("Save", filename, "to", wdir + out)

print('finish')



In [ ]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Working dir
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/"
file = "raa01-sf_10000-1701012350-dwd---bin"
#Read the original data: data = 2d Array(row,col) and metadata (productType, datetime,nodataflag)
#Origin in lower left corner
data_raw, meta  = wrl.io.read_radolan_composite(wdir + file)

#Round data to 2 decimal places to save memory
data_raw_s = np.around(data_raw, decimals = 2, out = None)

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)

#flip the data and the coords up-down (from lower left to upper left corner)
data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')

#Create raster dataset: Geotiff with radolan projection
ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)

#Reproject dataset to EPSG 3857
ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
#Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
#This line is needed until fixed
ds2.SetProjection(epsg_3857.ExportToWkt())

#write raster dataset
filename = file[15:25] + ".tif"
wrl.io.write_raster_dataset(wdir + filename, ds2, "GTiff")
#

In [ ]:
from osgeo import gdal
import numpy as np
raster = None
array = None
raster = gdal.Open("/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/out/0901022350.tif")
#band = raster.GetRasterBand(1)
array = raster.ReadAsArray()
print(np.median(array))


In [ ]:
# load radolan files
print(os.getcwd())
rw_filename = wrl.util.get_wradlib_data_file("raa01-sf_10000-0901012350-dwd---bin.gz")
ds, rwattrs = wrl.io.read_radolan_composite(rw_filename, loaddata='xarray')
# print the xarray dataset
print(ds)